# Store Sales Time Series Forecasting  
Group members: Spencer Potter, Ben Smith, Carter Watson

[Kaggle Data](https://www.kaggle.com/competitions/store-sales-time-series-forecasting)

**Imports**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from scipy.sparse import csr_matrix

**Introducing the DataSet**

In [2]:
train_df=pd.read_csv("needed_data/train.csv")
train_df.shape
test_df=pd.read_csv("needed_data/test.csv")
test_df.shape
print(train_df)

              id        date  store_nbr                      family     sales  \
0              0  2013-01-01          1                  AUTOMOTIVE     0.000   
1              1  2013-01-01          1                   BABY CARE     0.000   
2              2  2013-01-01          1                      BEAUTY     0.000   
3              3  2013-01-01          1                   BEVERAGES     0.000   
4              4  2013-01-01          1                       BOOKS     0.000   
...          ...         ...        ...                         ...       ...   
3000883  3000883  2017-08-15          9                     POULTRY   438.133   
3000884  3000884  2017-08-15          9              PREPARED FOODS   154.553   
3000885  3000885  2017-08-15          9                     PRODUCE  2419.729   
3000886  3000886  2017-08-15          9  SCHOOL AND OFFICE SUPPLIES   121.000   
3000887  3000887  2017-08-15          9                     SEAFOOD    16.000   

         onpromotion  
0   

In [3]:
train_df.sort_index()
train_df.set_index('id')
test_df.sort_index()
test_df.set_index('id')

,date,store_nbr,family,onpromotion
id,,,,
3000888,2017-08-16,1,AUTOMOTIVE,0
3000889,2017-08-16,1,BABY CARE,0
3000890,2017-08-16,1,BEAUTY,2
3000891,2017-08-16,1,BEVERAGES,20
3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...
3029395,2017-08-31,9,POULTRY,1
3029396,2017-08-31,9,PREPARED FOODS,0
3029397,2017-08-31,9,PRODUCE,1


**Data Splitting**

In [4]:
x_train = train_df.loc[:, train_df.columns != 'sales']
y_train = train_df['sales']
x_test = test_df.loc[:, test_df.columns != 'sales']

**Preprocessing Data**

*Create lag and windows for onpromotion*

In [5]:
# Creating lag features for the 'onpromotion' column for the past 3 days
for lag in range(1, 4):  # 3 lags
    x_train[f"onpromotion_lag_{lag}"] = x_train["onpromotion"].shift(lag)

# Creating rolling window average and sum for the 'onpromotion' column for the past 7 days
x_train["onpromotion_rolling_avg_7"] = x_train["onpromotion"].rolling(window=7).mean()
x_train["onpromotion_rolling_sum_7"] = x_train["onpromotion"].rolling(window=7).sum()

# Compute rate of change with a check for zero in the denominator
denominator = x_train["onpromotion"].shift()
numerator = x_train["onpromotion"].diff()
x_train["onpromotion_rate_of_change"] = np.where(denominator != 0, numerator / denominator, 0)

# Clean NaN's from data, replace with 0
lag_and_window_columns = [col for col in x_train.columns if col.startswith("onpromotion_")]
x_train[lag_and_window_columns] = x_train[lag_and_window_columns].fillna(0)

# Display the first few rows to inspect the newly created features
x_train.head(1135137)  # Displaying 1135137 rows to better see the populated rolling features (rare to see non zero/NaN values in the first few rows)

,id,date,store_nbr,family,onpromotion,onpromotion_lag_1,onpromotion_lag_2,onpromotion_lag_3,onpromotion_rolling_avg_7,onpromotion_rolling_sum_7,onpromotion_rate_of_change
0,0,2013-01-01,1,AUTOMOTIVE,0,0.0,0.0,0.0,0.000000,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0,0.0,0.0,0.0,0.000000,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0,0.0,0.0,0.0,0.000000,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0,0.0,0.0,0.0,0.000000,0.0,0.0
4,4,2013-01-01,1,BOOKS,0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1135132,1135132,2014-09-30,9,SCHOOL AND OFFICE SUPPLIES,0,125.0,1.0,1.0,18.142857,127.0,-1.0
1135133,1135133,2014-09-30,9,SEAFOOD,0,0.0,125.0,1.0,18.142857,127.0,0.0
1135134,1135134,2014-10-01,1,AUTOMOTIVE,0,0.0,0.0,125.0,18.142857,127.0,0.0
1135135,1135135,2014-10-01,1,BABY CARE,0,0.0,0.0,0.0,18.142857,127.0,0.0


*Create features for pay day and weekends*  
TODO: incorporate holidays and day they are actually celebrated  

In [6]:
# paydays are on the 15th and last day of the month
# add days since last payday
x_train['days_since_last_payday'] = np.where(pd.to_datetime(x_train['date']).dt.day <= 15, 
                                             pd.to_datetime(x_train['date']).dt.day - 1, 
                                             pd.to_datetime(x_train['date']).dt.day - 16)

# boolean for whether the date is a weekend
x_train['is_weekend'] = (pd.to_datetime(x_train['date']).dt.dayofweek >= 5).astype(int)

# Display the first few rows to inspect the feature
x_train[['date', 'days_since_last_payday', 'is_weekend']].drop_duplicates().head(20)  # Display unique dates for better visibility


,date,days_since_last_payday,is_weekend
0,2013-01-01,0,0
1782,2013-01-02,1,0
3564,2013-01-03,2,0
5346,2013-01-04,3,0
7128,2013-01-05,4,1
8910,2013-01-06,5,1
10692,2013-01-07,6,0
12474,2013-01-08,7,0
14256,2013-01-09,8,0
16038,2013-01-10,9,0


**Perform same column transforms and evaluations on x_test**

In [8]:
# Creating lag features for the 'onpromotion' column for the past 3 days
for lag in range(1, 4):  # 3 lags
    x_test[f"onpromotion_lag_{lag}"] = x_test["onpromotion"].shift(lag)

# Creating rolling window average and sum for the 'onpromotion' column for the past 7 days
x_test["onpromotion_rolling_avg_7"] = x_test["onpromotion"].rolling(window=7).mean()
x_test["onpromotion_rolling_sum_7"] = x_test["onpromotion"].rolling(window=7).sum()

# Compute rate of change with a check for zero in the denominator
denominator = x_test["onpromotion"].shift()
numerator = x_test["onpromotion"].diff()
x_test["onpromotion_rate_of_change"] = np.where(denominator != 0, numerator / denominator, 0)

# Clean NaN's from data, replace with 0
lag_and_window_columns = [col for col in x_test.columns if col.startswith("onpromotion_")]
x_test[lag_and_window_columns] = x_test[lag_and_window_columns].fillna(0)

# add days since last payday
x_test['days_since_last_payday'] = np.where(pd.to_datetime(x_test['date']).dt.day <= 15, 
                                             pd.to_datetime(x_test['date']).dt.day - 1, 
                                             pd.to_datetime(x_test['date']).dt.day - 16)

# boolean for whether the date is a weekend
x_test['is_weekend'] = (pd.to_datetime(x_test['date']).dt.dayofweek >= 5).astype(int)

# Display the first few rows to inspect the feature
x_test[['date', 'days_since_last_payday', 'is_weekend']].drop_duplicates().head(20)  # Display unique dates for better visibility


,date,days_since_last_payday,is_weekend
0,2017-08-16,0,0
1782,2017-08-17,1,0
3564,2017-08-18,2,0
5346,2017-08-19,3,1
7128,2017-08-20,4,1
8910,2017-08-21,5,0
10692,2017-08-22,6,0
12474,2017-08-23,7,0
14256,2017-08-24,8,0
16038,2017-08-25,9,0


In [9]:
numeric_data=["onpromotion", "store_nbr", "days_since_last_payday", "onpromotion_rolling_avg_7", "onpromotion_rolling_sum_7", "onpromotion_rate_of_change"]
for i in range(1, 4):
    numeric_data.append(f"onpromotion_lag_{i}")

categorical_features = ["family"]
boolean_features = ["is_weekend"]
passthrough_data=[] + boolean_features
drop_data = ["id", "date"]

**Pipeline Creation**

In [10]:
preprocessor = make_column_transformer(
    (OneHotEncoder(handle_unknown="ignore"), categorical_features),
    (StandardScaler(), numeric_data),
    ("passthrough", passthrough_data),
    ("drop", drop_data),
)

In [11]:
x_train_transformed = preprocessor.fit_transform(x_train)
x_train_transformed.shape

(3000888, 43)

## Dummy Model

In [12]:
baseline_model = make_pipeline(preprocessor, DummyRegressor(strategy='mean'))

In [13]:
results_dict = {}
scores = cross_val_score(baseline_model, x_train, y_train, cv=10)
results_dict['cross_val_scores'] = -scores
results_df = pd.DataFrame(results_dict)
print(results_df)

#Because we're using the dummy regressor, smaller values mean better accuracy

   cross_val_scores
0          0.062360
1          0.045408
2          0.006066
3          0.002078
4          0.001458
5          0.001344
6          0.005386
7          0.004660
8          0.008070
9          0.010618


In [14]:
baseline_model.fit(x_train, y_train)
prediction = baseline_model.predict(x_test)
prediction_df = pd.DataFrame({'id': x_test.id, 'sales': prediction})
prediction_df.to_csv("dummy_test_predictions.csv", index=False)

## Linear Regression Model

In [15]:
from sklearn.linear_model import LinearRegression
lRModel = make_pipeline(preprocessor, LinearRegression())
results_lR_dict = {}
scores = cross_val_score(lRModel, x_train, y_train, cv=10)
results_lR_dict['cross_val_scores'] = scores
results_lR_df = pd.DataFrame(results_lR_dict)
print(results_lR_df)

   cross_val_scores
0          0.308628
1          0.394058
2          0.462146
3          0.501434
4          0.510904
5          0.551055
6          0.552296
7          0.535783
8          0.578457
9          0.597075


In [16]:
lRModel.fit(x_train, y_train)
lR_prediciton=lRModel.predict(x_test)
lR_prediciton = [max(0, pred) for pred in lR_prediciton]
prediction_df = pd.DataFrame({'id': x_test.id, 'sales': lR_prediciton})
prediction_df.to_csv("linear_regression_test_predictions.csv", index=False)

## 